Hampshire Schools Data
-------------------------

Shows capacity data for schools in Hampshire, Southampton and Portsmouth.

Dependencies:
* Numpy (pip install numpy) (Hint: remove distro-provided version...)
* Panda (pip install panda)
* ipyleaflet (pip install ipyleaflet)
* convertbng (pip install convertbng)

In [7]:
import pandas as pd
import numpy as np

In [8]:
# Load some data
d_sch = pd.read_csv('hampshire_schools_dec2018.csv', quotechar='"');
#display(d_sch); # Prints the whole data table
display(d_sch.columns.values) # Prints column names

d_sch = d_sch[np.isfinite(d_sch['SchoolCapacity'])] # Filter to include only schools with capacity data
d_sch = d_sch[np.isfinite(d_sch['NumberOfPupils'])]


array(['URN', 'LA (code)', 'LA (name)', 'EstablishmentNumber',
       'EstablishmentName', 'TypeOfEstablishment (name)',
       'EstablishmentTypeGroup (name)', 'EstablishmentStatus (name)',
       'ReasonEstablishmentOpened (name)', 'OpenDate',
       'ReasonEstablishmentClosed (name)', 'CloseDate',
       'PhaseOfEducation (name)', 'StatutoryLowAge', 'StatutoryHighAge',
       'Boarders (name)', 'NurseryProvision (name)',
       'OfficialSixthForm (name)', 'Gender (name)',
       'ReligiousCharacter (name)', 'ReligiousEthos (name)',
       'Diocese (name)', 'AdmissionsPolicy (name)', 'SchoolCapacity',
       'SpecialClasses (name)', 'CensusDate', 'NumberOfPupils',
       'NumberOfBoys', 'NumberOfGirls', 'PercentageFSM',
       'TrustSchoolFlag (name)', 'Trusts (name)',
       'SchoolSponsorFlag (name)', 'SchoolSponsors (name)',
       'FederationFlag (name)', 'Federations (name)', 'UKPRN',
       'FEHEIdentifier', 'FurtherEducationType (name)', 'OfstedLastInsp',
       'Street', 'Loc

In [9]:
d_sch['Utilisation'] = d_sch.NumberOfPupils / d_sch.SchoolCapacity;
d_sch['FreePlaces'] = d_sch['SchoolCapacity'] - d_sch['NumberOfPupils'];

#display(d_sch);

In [14]:
from ipyleaflet import Map, Marker, CircleMarker, Popup, basemaps, basemap_to_tiles
from ipywidgets import HTML
from convertbng.util import convert_lonlat

# You need to: `jupyter nbextension enable --py --sys-prefix ipyleaflet`
# after installing ipyleaflet...

c = (50.970700, -1.351545) # Eastleigh House

m = Map(
    center=c,
    zoom=12,
    dragging=True
)


def genInfo(school):
    i = HTML()
    i.value = "<strong>" + row.loc['EstablishmentName'] + "</strong><br /><table>" + \
    "<tr><td>Phase:</td><td>" + row.loc['PhaseOfEducation (name)'] + " " + str(int(row.loc['StatutoryLowAge'])) + '-' + str(int(row.loc['StatutoryHighAge'])) + "</td></tr>" + \
    "<tr><td>Nominal Capacity:</td><td>" + str(int(row.loc['SchoolCapacity'])) + "</td></tr>" + \
    "<tr><td>Current Pupils:</td><td>" + str(int(row.loc['NumberOfPupils'])) + "</td></tr>" + \
    "<tr style=\"border-top: 1px solid #444;\"><td>Free Places:</td><td>" + str(int(row.loc['FreePlaces'])) + "</td></tr>"
    "</table>"
    
    return i

def pccolour(util):
    
    if(util < 0.95):
        return '#9bf442';
    if(util <= 1):
        return '#f4d742';
    if(util < 1.05):
        return '#f4ad42';
    else:
        return '#f45c42';


for index, row in d_sch.iterrows():
    lon, lat = convert_lonlat([row.loc['Easting']], [row.loc['Northing']])
    loc = (lat[0], lon[0])
    marker = CircleMarker(location=loc, radius=4)
    marker.color = pccolour(row.loc['Utilisation']);
    m.add_layer(marker)
    
    # Create info popup
    message = genInfo(row)
    marker.popup = message


m

Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…